# News Classification using LSTM

### Dataset : https://www.kaggle.com/datasets/sulphatet/twitter-financial-news

### Labels Chosen:
* Product News(0) and Stock Commentary(1)


## Importing the libraries

In [ ]:
import pandas as pd
import tensorflow as tf
import nltk
import re
import numpy as np

In [ ]:
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense, Dropout

## Loading the data

In [ ]:
data =pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DEEP LEARNING/classification_train_data.csv")
data.head()

,text,label
0,$HOUR flagging here below the squeeze level to...,1
1,$SPY closed just above 2 mo channel &amp; 10d ...,1
2,$VLCN going green.....,1
3,$QQQ - QQQ: It's Make It Or Break It For The S...,1
4,Nike college apparel will have 'faster speed t...,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5663 entries, 0 to 5662
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5663 non-null   object
 1   label   5663 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 88.6+ KB


In [ ]:
data.isnull().sum()

text     0
label    0
dtype: int64

## Defining the X and Y features

In [ ]:
X = data.drop("label", axis =1)
y = data["label"]

In [ ]:
X.shape, y.shape

((5663, 1), (5663,))

## Defining the vocabulary size

In [ ]:
vocab_size = 5000

In [ ]:
news = X.copy()

## Data Preprocessing

In [ ]:
corpus= []
def data_preprocess(news):
  ps = PorterStemmer()

  for i in range(0, len(news)):
    text = re.sub("[^a-zA-Z]", " ", news["text"][i])
    text = text.lower()
    text= text.split()
    text = [ps.stem(word) for word in text if not word in stopwords.words("english")]
    text = " ".join(text)
    corpus.append(text)
data_preprocess(news)

## One-Hot Encoding

In [ ]:
one_hot_rep =[]
def One_Hot(corpus):
  for words in corpus:
    onehot = one_hot(words, vocab_size)
    one_hot_rep.append(onehot)
  # one_hot_rep = [one_hot(words, vocab_size) for words in corpus]
One_Hot(corpus)

In [ ]:
maxlen=-1
for doc in corpus:
    tokens=nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen=len(tokens)
print("The maximum number of words in any document is : ",maxlen)

The maximum number of words in any document is :  50


## Generating the padded sequences

In [ ]:
# The one hot encoded sequences are of unequal length so we do padding.
sent_length = 20
embedded_docs = pad_sequences(one_hot_rep, padding="pre", maxlen =sent_length)
embedded_docs[0]

array([   0,    0,    0,    0,    0, 1529, 1543,  675, 2543, 1329,  979,
        381, 1541, 4302,  776,  582, 3450, 2688,  675, 3422], dtype=int32)

## Model Building

In [ ]:
embedding_vector_features = 40
model =Sequential()
model.add(Embedding(vocab_size,embedding_vector_features, input_length =sent_length ))
model.add(LSTM(100))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Converting the padded sequences into an array
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
X_final.shape, y_final.shape

((5663, 20), (5663,))

In [ ]:
X_train,X_test, y_train,y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=42)

In [ ]:
history =model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size =50)

Epoch 1/10
91/91 [==============================] - 6s 39ms/step - loss: 0.4151 - accuracy: 0.8159 - val_loss: 0.1791 - val_accuracy: 0.9470
Epoch 2/10
91/91 [==============================] - 3s 32ms/step - loss: 0.1189 - accuracy: 0.9585 - val_loss: 0.1518 - val_accuracy: 0.9479
Epoch 3/10
91/91 [==============================] - 5s 57ms/step - loss: 0.0548 - accuracy: 0.9859 - val_loss: 0.1823 - val_accuracy: 0.9365
Epoch 4/10
91/91 [==============================] - 6s 71ms/step - loss: 0.0254 - accuracy: 0.9951 - val_loss: 0.2293 - val_accuracy: 0.9267
Epoch 5/10
91/91 [==============================] - 5s 55ms/step - loss: 0.0137 - accuracy: 0.9974 - val_loss: 0.2362 - val_accuracy: 0.9285
Epoch 6/10
91/91 [==============================] - 3s 32ms/step - loss: 0.0109 - accuracy: 0.9980 - val_loss: 0.3115 - val_accuracy: 0.9259
Epoch 7/10
91/91 [==============================] - 3s 32ms/step - loss: 0.0059 - accuracy: 0.9989 - val_loss: 0.3412 - val_accuracy: 0.9250
Epoch 8/10
91

## Model Evaluation

In [ ]:
model.evaluate(X_train, y_train)

142/142 [==============================] - 1s 7ms/step - loss: 0.0065 - accuracy: 0.9969


[0.00653222668915987, 0.996909499168396]

In [ ]:
y_pred= model.predict(X_test)

36/36 [==============================] - 1s 7ms/step


In [ ]:
y_pred = np.round(y_pred)
y_pred

array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss: ", round(loss,2), "Accuracy: ","%.2f"%accuracy)

36/36 [==============================] - 0s 7ms/step - loss: 0.4623 - accuracy: 0.9117
Loss:  0.46 Accuracy:  0.91


In [ ]:
print("Actual Value: ", y_test[0])
print("Predicted Value: ", y_pred[0])

Actual Value:  1
Predicted Value:  [1.]


## Adding Dropout Layer

In [ ]:
model1 =Sequential()
model1.add(Embedding(vocab_size,embedding_vector_features, input_length =sent_length ))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1, activation="sigmoid"))
model1.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model1.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size =50)

Epoch 1/10
91/91 [==============================] - 6s 39ms/step - loss: 0.4238 - accuracy: 0.8011 - val_loss: 0.2115 - val_accuracy: 0.9312
Epoch 2/10
91/91 [==============================] - 3s 33ms/step - loss: 0.1276 - accuracy: 0.9539 - val_loss: 0.1561 - val_accuracy: 0.9435
Epoch 3/10
91/91 [==============================] - 3s 34ms/step - loss: 0.0645 - accuracy: 0.9797 - val_loss: 0.1635 - val_accuracy: 0.9426
Epoch 4/10
91/91 [==============================] - 4s 48ms/step - loss: 0.0359 - accuracy: 0.9894 - val_loss: 0.2490 - val_accuracy: 0.9391
Epoch 5/10
91/91 [==============================] - 3s 37ms/step - loss: 0.0228 - accuracy: 0.9940 - val_loss: 0.1936 - val_accuracy: 0.9365
Epoch 6/10
91/91 [==============================] - 3s 33ms/step - loss: 0.0128 - accuracy: 0.9974 - val_loss: 0.3574 - val_accuracy: 0.9162
Epoch 7/10
91/91 [==============================] - 3s 33ms/step - loss: 0.0123 - accuracy: 0.9978 - val_loss: 0.3218 - val_accuracy: 0.9329
Epoch 8/10
91

In [ ]:
loss, accuracy = model1.evaluate(X_test, y_test)
print("Loss: ", round(loss,2), "Accuracy: ","%.2f"%accuracy)

36/36 [==============================] - 0s 7ms/step - loss: 0.4121 - accuracy: 0.9214
Loss:  0.41 Accuracy:  0.92


## Model Testing

In [ ]:
test_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DEEP LEARNING/classification_test_data.csv")
test_data.head()

,text,label
0,https://t.co/G0nmNITjcy is cutting 25% of its...,0
1,https://t.co/XNB7m39H8H Launches Fixed-term S...,0
2,#ALSI future monthly putting in a reversal. St...,1
3,#ALSI major constituent weightings within Top4...,1
4,#Consumerdiscretionary outperforms $XLY $TSLA ...,1


In [ ]:
test_data.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
info =test_data.drop("label", axis=1)
info

,text
0,https://t.co/G0nmNITjcy is cutting 25% of its...
1,https://t.co/XNB7m39H8H Launches Fixed-term S...
2,#ALSI future monthly putting in a reversal. St...
3,#ALSI major constituent weightings within Top4...
4,#Consumerdiscretionary outperforms $XLY $TSLA ...
...,...
1375,ZapBatt Partners with Toshiba to Unlock Proven...
1376,ZEDEDA Closes $26M Series B Funding Round as D...
1377,Zultys Receives 2022 Unified Communications Pr...
1378,Zymeworks Announces Plan to Become a Delaware ...


In [ ]:
corpus1= []
def data_preprocess(news):
  ps = PorterStemmer()

  for i in range(0, len(news)):
    text = re.sub("[^a-zA-Z]", " ", news["text"][i])
    text = text.lower()
    text= text.split()
    text = [ps.stem(word) for word in text if not word in stopwords.words("english")]
    text = " ".join(text)
    corpus1.append(text)
data_preprocess(info)

In [ ]:
one_hot_repr = [one_hot(words, vocab_size) for words in corpus1]
embedded_docu = pad_sequences(one_hot_repr, padding="pre", maxlen =sent_length)
embedded_docu[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4199,
       1642, 4242, 3290,  515, 3707, 2808, 4429, 3417, 1863], dtype=int32)

In [ ]:
prediction = model.predict(embedded_docu)
prediction = np.round(prediction)
print(prediction)

44/44 [==============================] - 0s 7ms/step
[[1.]
 [0.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]]


In [ ]:
print("Actual Value: ", y_test[0])
print("Predicted Value: ", prediction[0])


Actual Value:  1
Predicted Value:  [1.]


In [ ]:
loss, accuracy = model.evaluate(embedded_docu)
print("Loss: ", round(loss,2), "Accuracy: ","%.2f"%accuracy)

44/44 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Loss:  0.0 Accuracy:  0.00


# Conclusion:
* The results obtained for both RNN and LSTM is the same.
*  We have used the News Classification dataset for our LSTM model. We have preprocessed the data and one hot encoded the values.As the sequence of input to our model should be of equal length we pad them. We have set the input dimensions as 5000 and the length of the input sequence is 20. We will be embedding the 5000 word vocabulary into 40 dimension (embedding_feature_vector). The model consists of 1 embedding, 1 RNN layer and an output layer. The accuracy of this model is 99%. We obtained the same results even after adding the batch normalization layer.

* The model is tested with a new dataset and though the model was able to predict  both the  classes, it's prediction is completely wrong.
